#Import + mount Google drive

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np

import time

import math

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

import sys
from google.colab import drive
import importlib.util

# Mounting Google Drive 
drive.mount('/content/gdrive')


#use GPU is available
use_gpu = torch.cuda.is_available()



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#Groupy loading and installation

In [0]:
!pip install nose
!pip install chainer

%cd /content/gdrive/My Drive/Deep Learning/Reproduction project
! git clone https://github.com/adambielski/GrouPy.git

%cd /content/gdrive/My Drive/Deep Learning/Reproduction project/GrouPy
! python setup.py install

!nosetests -v

from groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4, P4MConvZ2, P4MConvP4M

     |████████████████████████████████| 163kB 6.5MB/s 
/content/gdrive/My Drive/Deep Learning/Reproduction project
fatal: destination path 'GrouPy' already exists and is not an empty directory.
/content/gdrive/My Drive/Deep Learning/Reproduction project/GrouPy
running install
running build
running build_py
running install_lib
creating /usr/local/lib/python3.6/dist-packages/groupy
copying build/lib/groupy/__init__.py -> /usr/local/lib/python3.6/dist-packages/groupy
creating /usr/local/lib/python3.6/dist-packages/groupy/garray
copying build/lib/groupy/garray/test_garray.py -> /usr/local/lib/python3.6/dist-packages/groupy/garray
copying build/lib/groupy/garray/garray.py -> /usr/local/lib/python3.6/dist-packages/groupy/garray
copying build/lib/groupy/garray/__init__.py -> /usr/local/lib/python3.6/dist-packages/groupy/garray
copying build/lib/groupy/garray/Z2_array.py -> /usr/local/lib/python3.6/dist-packages/groupy/garray
copying build/lib/groupy/garray/finitegroup.py -> /usr/local/lib/pyt

#Residual Network: Group Equivariant Convolutional Networks

In [0]:
# ---------------------------- RESHAPE TENSORS ---------------------------------
def reshape_5Dto4D(x):
    xs = x.size()
    x = x.view(xs[0], xs[1]*xs[2], xs[3], xs[4])
    return xs, x
    
def reshape_4Dto5D(xs, x):
    x = x.view(xs[0], xs[1], xs[2], x.size()[2], x.size()[3])
    return x


# ------------------------------- BASIC BLOCK ----------------------------------
class Basic_Block_P4(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1, 
                 *args,**kwargs):
        super(Basic_Block_P4, self).__init__()
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.downsampling = downsampling

        #convolutions and batch-normalizations
        self.g_conv1 = P4ConvP4(in_channels, out_channels, kernel_size=3, 
                                bias=False, padding = 1, stride = downsampling)
        self.bn1     = nn.BatchNorm2d(out_channels*4)
        self.g_conv2 = P4ConvP4(out_channels, out_channels, kernel_size=3, 
                                bias=False, padding = 1)
        self.bn2     = nn.BatchNorm2d(out_channels*4)

        #shortcut connections
        self.id_shortcut   = nn.Sequential()
        self.shortcut_conv = P4ConvP4(in_channels, out_channels, kernel_size=1, 
                                      bias=False, stride = downsampling)
        self.shortcut_bn   = nn.BatchNorm2d(out_channels*4)

    def forward(self, x):
        #convolutions, activations and batch-normalizations
        out = self.g_conv1(x)
        outs, out = reshape_5Dto4D(out)
        out = F.relu(self.bn1(out))
        out = reshape_4Dto5D(outs, out)
        out = self.g_conv2(out)
        outs, out = reshape_5Dto4D(out)
        out = self.bn2(out)

        if self.downsampling == 1 and self.in_channels == self.out_channels:
            x_shorts, x_short = reshape_5Dto4D(x)
            x_short = self.id_shortcut(x_short)
        elif self.downsampling != 1 or self.in_channels != self.out_channels:
            x_short = self.shortcut_conv(x)
            x_shorts, x_short = reshape_5Dto4D(x_short)
            x_short = self.shortcut_bn(x_short)


        #adding both up
        out += x_short
        out = reshape_4Dto5D(outs, F.relu(out))

        return out

# -------------------------------- INIT BLOCK ----------------------------------
class Init_Block_P4(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1, 
                 *args,**kwargs):
        super(Init_Block_P4, self).__init__()
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.downsampling = downsampling

        #convolutions and batch-normalizations
        self.g_conv1 = P4ConvZ2(in_channels, out_channels, kernel_size=7, 
                                bias=False, padding = 3, stride = downsampling)
        self.bn1     = nn.BatchNorm2d(out_channels*4)

    def forward(self, x):
        #convolutions, activations and batch-normalizations
        out = self.g_conv1(x)
        outs, out = reshape_5Dto4D(out)
        out = F.relu(self.bn1(out))
        out = reshape_4Dto5D(outs, out)
        return out

# ------------------------- ENCODER OF THE NETWORK -----------------------------
class ResNet_P4(nn.Module):
    '''
    This class represents the full implementation of the residual network.
    '''
    def __init__(self, in_channels, n_classes, 
                 init_block = Init_Block_P4, resblock = Basic_Block_P4, 
                 blocks = [16, 32, 64], depths = [7,7,7], *args, **kwargs):
        super(ResNet_P4,self).__init__()

        
        #Encoder part
        self.init_conv = Init_Block_P4(in_channels, blocks[0])
        self.layer1    = self._residual_layer(blocks[0], blocks[0], resblock, depths[0])
        self.layer2    = self._residual_layer(blocks[0], blocks[1], resblock, depths[1])
        self.layer3    = self._residual_layer(blocks[1], blocks[2], resblock, depths[2])

        #Decoder part
        self.avg_pool = nn.AvgPool2d(4)
        self.linear1  = nn.Linear(blocks[2]*64*4, n_classes)


    def _residual_layer(self, in_channels, out_channels, block, num_blocks, downsampling = 2):
        layers = [block(in_channels, out_channels, stride = downsampling)]
        for n in range(num_blocks-1):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        #encoder
        out = self.init_conv(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        outs, out = reshape_5Dto4D(out)

        #decoder part
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        return out



# -------------------------- TESING OF THE NETWORK -----------------------------
def resnet44_P4(in_channels, n_classes):
    return ResNet_P4(in_channels, n_classes)


##Checking Time

In [0]:
# ---------------------------- RESHAPE TENSORS ---------------------------------
def reshape_5Dto4D(x):
    xs = x.size()
    x = x.view(xs[0], xs[1]*xs[2], xs[3], xs[4])
    return xs, x
    
def reshape_4Dto5D(xs, x):
    x = x.view(xs[0], xs[1], xs[2], x.size()[2], x.size()[3])
    return x


# ------------------------------- BASIC BLOCK ----------------------------------
class Basic_Block_P4(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1, 
                 *args,**kwargs):
        super(Basic_Block_P4, self).__init__()
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.downsampling = downsampling

        #convolutions and batch-normalizations
        self.g_conv1 = P4ConvP4(in_channels, out_channels, kernel_size=3, 
                                bias=False, padding = 1, stride = downsampling)
        self.bn1     = nn.BatchNorm2d(out_channels*4)
        self.g_conv2 = P4ConvP4(out_channels, out_channels, kernel_size=3, 
                                bias=False, padding = 1)
        self.bn2     = nn.BatchNorm2d(out_channels*4)

        #shortcut connections
        self.id_shortcut   = nn.Sequential()
        self.shortcut_conv = P4ConvP4(in_channels, out_channels, kernel_size=1, 
                                      bias=False, stride = downsampling)
        self.shortcut_bn   = nn.BatchNorm2d(out_channels*4)

    def forward(self, x):
        print('   NEW SUB AREA: BASIC BLOCK')
        start = time.time()
        #convolutions, activations and batch-normalizations
        out = self.g_conv1(x)
        print('   Time check after 1st g_conv:')
        print(time.time()-start)
        start = time.time()
        outs, out = reshape_5Dto4D(out)
        print('   Time check after 1st reshape:')
        print(time.time()-start)
        out = F.relu(self.bn1(out))
        out = reshape_4Dto5D(outs, out)
        start = time.time()
        out = self.g_conv2(out)
        print('   Time check after 2nd g_conv:')
        print(time.time()-start)
        start = time.time()
        outs, out = reshape_5Dto4D(out)
        print('   Time check after 2nd reshape:')
        print(time.time()-start)
        start = time.time()
        out = self.bn2(out)
        print('   Time check after 2nd batchnorm:')
        print(time.time()-start)

        if self.downsampling == 1 and self.in_channels == self.out_channels:
            start = time.time()
            x_shorts, x_short = reshape_5Dto4D(x)
            print('   Time check after shortcut reshape:')
            print(time.time()-start)
            x_short = self.id_shortcut(x_short)
        elif self.downsampling != 1 or self.in_channels != self.out_channels:
            start = time.time()
            x_short = self.shortcut_conv(x)
            print('   Time check after 1x1 g_conv:')
            print(time.time()-start)
            start = time.time()
            x_shorts, x_short = reshape_5Dto4D(x_short)
            print('   Time check after shortcut reshape:')
            print(time.time()-start)
            start = time.time()
            x_short = self.shortcut_bn(x_short)
            print('   Time check after shortcut batchnorm:')
            print(time.time()-start)

        #adding both up
        out += x_short
        out = reshape_4Dto5D(outs, F.relu(out))
        print('   Time check after reshape shortcut + convolution branch:')
        print(time.time()-start)

        return out

# -------------------------------- INIT BLOCK ----------------------------------
class Init_Block_P4(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1, 
                 *args,**kwargs):
        super(Init_Block_P4, self).__init__()
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.downsampling = downsampling

        #convolutions and batch-normalizations
        self.g_conv1 = P4ConvZ2(in_channels, out_channels, kernel_size=7, 
                                bias=False, padding = 3, stride = downsampling)
        self.bn1     = nn.BatchNorm2d(out_channels*4)

    def forward(self, x):
        #convolutions, activations and batch-normalizations
        print('   NEW SUB AREA: INIT BLOCK')
        start = time.time()
        out = self.g_conv1(x)
        print('   Time check after inital g_conv:')
        print(time.time()-start)
        start = time.time()
        outs, out = reshape_5Dto4D(out)
        print('   Time check after reshape 5 --> 4:')
        print(time.time()-start)
        start = time.time()
        out = F.relu(self.bn1(out))
        print('   Time check after batch norm:')
        print(time.time()-start)
        start = time.time()
        out = reshape_4Dto5D(outs, out)
        print('   Time check after reshape 4 --> 5:')
        print(time.time()-start)
        return out

# ------------------------- ENCODER OF THE NETWORK -----------------------------
class ResNet_P4(nn.Module):
    '''
    This class represents the full implementation of the residual network.
    '''
    def __init__(self, in_channels, n_classes, 
                 init_block = Init_Block_P4, resblock = Basic_Block_P4, 
                 blocks = [16, 32, 64], depths = [7,7,7], *args, **kwargs):
        super(ResNet_P4,self).__init__()

        
        #Encoder part
        self.init_conv = Init_Block_P4(in_channels, blocks[0])
        self.layer1    = self._residual_layer(blocks[0], blocks[0], resblock, depths[0])
        self.layer2    = self._residual_layer(blocks[0], blocks[1], resblock, depths[1])
        self.layer3    = self._residual_layer(blocks[1], blocks[2], resblock, depths[2])

        #Decoder part
        self.avg_pool = nn.AvgPool2d(4)
        self.linear1  = nn.Linear(blocks[2]*64*4, n_classes)


    def _residual_layer(self, in_channels, out_channels, block, num_blocks, downsampling = 2):
        layers = [block(in_channels, out_channels, stride = downsampling)]
        for n in range(num_blocks-1):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        #encoder
        print('NEW AREA: INIT BLOCK')
        start_whole = time.time()
        out = self.init_conv(x)
        print('Time check after initial convolution layer:')
        print(time.time()-start_whole)
        print('')
        print('NEW AREA: LAYER 1')
        start_whole = time.time()
        out = self.layer1(out)
        print('Time check after layer 1:')
        print(time.time()-start_whole)
        print('')
        print('NEW AREA: LAYER 2')
        start_whole = time.time()
        out = self.layer2(out)
        print('Time check after layer 2:')
        print(time.time()-start_whole)
        print('')
        print('NEW AREA: LAYER 3')
        start_whole = time.time()
        out = self.layer3(out)
        print('Time check after layer 3:')
        print(time.time()-start_whole)
        start_whole = time.time()
        outs, out = reshape_5Dto4D(out)
        print('Time check after final reshape:')
        print(time.time()-start_whole)

        #decoder part
        print('')
        print('NEW AREA: FULLY CONNECTED')
        start_whole = time.time()
        out = self.avg_pool(out)
        print(' Time check after final average pooling:')
        print(time.time()-start_whole)
        out = out.view(out.size(0), -1)
        start_whole = time.time()
        out = self.linear1(out)
        print(' Time check final linear layer:')
        print(time.time()-start_whole)
        return out



# -------------------------- TESING OF THE NETWORK -----------------------------
def resnet44_P4(in_channels, n_classes):
    return ResNet_P4(in_channels, n_classes)

In [0]:
def resnet_p4_test(in_channels, out_channels):
    return ResNet_P4(in_channels, out_channels)

resnet_p4_test = resnet_p4_test(3, 10)

evaluate_accuracy(testloader, resnet_p4_test)

NEW AREA: INIT BLOCK
   NEW SUB AREA: INIT BLOCK
   Time check after inital g_conv:
0.13975310325622559
   Time check after reshape 5 --> 4:
9.584426879882812e-05
   Time check after batch norm:
0.05186009407043457
   Time check after reshape 4 --> 5:
0.0007143020629882812
Time check after initial convolution layer:
0.19375944137573242

NEW AREA: LAYER 1
   NEW SUB AREA: BASIC BLOCK
   Time check after 1st g_conv:
0.19710946083068848
   Time check after 1st reshape:
0.0005748271942138672
   Time check after 2nd g_conv:
0.1714458465576172
   Time check after 2nd reshape:
0.0007448196411132812
   Time check after 2nd batchnorm:
0.011172771453857422
   Time check after shortcut reshape:
0.0003726482391357422
   Time check after reshape shortcut + convolution branch:
0.018550634384155273
   NEW SUB AREA: BASIC BLOCK
   Time check after 1st g_conv:
0.15905189514160156
   Time check after 1st reshape:
0.00043654441833496094
   Time check after 2nd g_conv:
0.16347217559814453
   Time check af

KeyboardInterrupt: ignored

##P4 and P4M integrated

In [0]:
# ---------------------------- RESHAPE TENSORS ---------------------------------
def reshape_5Dto4D(x):
    xs = x.size()
    x = x.view(xs[0], xs[1]*xs[2], xs[3], xs[4])
    return xs, x
    
def reshape_4Dto5D(xs, x):
    x = x.view(xs[0], xs[1], xs[2], x.size()[2], x.size()[3])
    return x


# ------------------------------- BASIC BLOCK ----------------------------------
class Basic_Block_G(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1, 
                 num_trans = 4, g_conv = P4ConvP4, *args,**kwargs):
        super(Basic_Block_G, self).__init__()
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.downsampling = downsampling
        self.num_trans    = num_trans

        #convolutions and batch-normalizations
        self.g_conv1 = g_conv(in_channels, out_channels, kernel_size=3, 
                                bias=False, padding = 1, stride = downsampling)
        self.bn1     = nn.BatchNorm2d(out_channels*num_trans)
        self.g_conv2 = g_conv(out_channels, out_channels, kernel_size=3, 
                                bias=False, padding = 1)
        self.bn2     = nn.BatchNorm2d(out_channels*num_trans)

        #shortcut connections
        self.id_shortcut   = nn.Sequential()
        self.shortcut_conv = g_conv(in_channels, out_channels, kernel_size=1, 
                                      bias=False, stride = downsampling)
        self.shortcut_bn   = nn.BatchNorm2d(out_channels*num_trans)

    def forward(self, x):
        #convolutions, activations and batch-normalizations
        out = self.g_conv1(x)
        outs, out = reshape_5Dto4D(out)
        out = F.relu(self.bn1(out))
        out = reshape_4Dto5D(outs, out)
        out = self.g_conv2(out)
        outs, out = reshape_5Dto4D(out)
        out = self.bn2(out)

        if self.downsampling == 1 and self.in_channels == self.out_channels:
            x_shorts, x_short = reshape_5Dto4D(x)
            x_short = self.id_shortcut(x_short)
        elif self.downsampling != 1 or self.in_channels != self.out_channels:
            x_short = self.shortcut_conv(x)
            x_shorts, x_short = reshape_5Dto4D(x_short)
            x_short = self.shortcut_bn(x_short)


        #adding both up
        out += x_short
        out = reshape_4Dto5D(outs, F.relu(out))

        return out

# -------------------------------- INIT BLOCK ----------------------------------
class Init_Block_G(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1,
                 num_trans = 4, g_conv = P4ConvP4,
                 *args,**kwargs):
        super(Init_Block_G, self).__init__()
        self.in_channels  = in_channels
        self.out_channels = out_channels
        self.downsampling = downsampling

        #convolutions and batch-normalizations
        self.g_conv1 = g_conv(in_channels, out_channels, kernel_size=7, 
                                bias=False, padding = 3, stride = downsampling)
        self.bn1     = nn.BatchNorm2d(out_channels*num_trans)

    def forward(self, x):
        #convolutions, activations and batch-normalizations
        out = self.g_conv1(x)
        outs, out = reshape_5Dto4D(out)
        out = F.relu(self.bn1(out))
        out = reshape_4Dto5D(outs, out)
        return out

# ------------------------- ENCODER OF THE NETWORK -----------------------------
class ResNet_G(nn.Module):
    '''
    This class represents the full implementation of the residual network.
    '''
    def __init__(self, in_channels, n_classes, 
                 init_block = Init_Block_G, resblock = Basic_Block_G, 
                 blocks = [16, 32, 64], depths = [7,7,7],
                 num_trans = 4, *args, **kwargs):
        super(ResNet_G,self).__init__()
        self.num_trans = num_trans

        #Chose between symmetry groups
        if self.num_trans == 4:
            self.g_conv      = P4ConvP4
            self.g_conv_init = P4ConvZ2
        elif self.num_trans == 8:
            self.g_conv      = P4MConvP4M
            self.g_conv_init = P4MConvZ2

        
        #Encoder part
        self.init_conv = Init_Block_G(in_channels, blocks[0], 
                                      num_trans = self.num_trans, g_conv = self.g_conv_init)
        self.layer1    = self._residual_layer(blocks[0], blocks[0], resblock, depths[0])
        self.layer2    = self._residual_layer(blocks[0], blocks[1], resblock, depths[1])
        self.layer3    = self._residual_layer(blocks[1], blocks[2], resblock, depths[2])

        #Decoder part
        self.avg_pool = nn.AvgPool2d(4)
        self.linear1  = nn.Linear(blocks[2]*64*self.num_trans, n_classes)


    def _residual_layer(self, in_channels, out_channels, block, num_blocks, downsampling = 2):
        layers = [block(in_channels, out_channels, stride = downsampling, 
                                num_trans = self.num_trans, g_conv = self.g_conv)]
        for n in range(num_blocks-1):
            layers.append(block(out_channels, out_channels, 
                                num_trans = self.num_trans, g_conv = self.g_conv))
        return nn.Sequential(*layers)

    def forward(self, x):
        #encoder
        out = self.init_conv(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        outs, out = reshape_5Dto4D(out)

        #decoder part
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        return out



# -------------------------- TESING OF THE NETWORK -----------------------------
def resnet44_P4(in_channels, n_classes):
    return ResNet_G(in_channels, n_classes)

def resnet44_P4M(in_channels, n_classes):
    return ResNet_G(in_channels, n_classes, blocks = [11, 23, 45], num_trans = 8)

##Faster implementation

In [0]:
from torch.autograd import Variable

# ---------------------------- RESHAPE TENSORS ---------------------------------
def reshape_5Dto4D(x):
    xs = x.size()
    x = x.view(xs[0], xs[1]*xs[2], xs[3], xs[4])
    return xs, x
    
def reshape_4Dto5D(xs, x):
    x = x.view(xs[0], xs[1], xs[2], x.size()[2], x.size()[3])
    return x




# --------------------------- CLASS RESHAPE ------------------------------------
#4D to 5D
class Reshape_4Dto5D(nn.Module):
    def __init__(self, xs):
        super().__init__()
        self.xs = xs

    def forward(self,x):
        x = x.view(xs[0], xs[1], xs[2], x.size()[0], x.size()[1])
        return x

#5D to 4D
class Reshape_5Dto4D(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
        xs = x.size()
        x = x.view(xs[0], xs[1]*xs[2], xs[3], xs[4])
        return xs, x



# -------------------------- G-CONV within nn.Module ---------------------------
class G_P4P4(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1, bias = False, 
                 padding = 1):
        super().__init__()
        self.g_conv = P4ConvP4(in_channels, out_channels, stride = downsampling, 
                               bias = bias, padding = padding)

    def forward(self,x):
        x = self.g_conv(x)
        return x

class G_P4Z2(nn.Module):
    def __init__(self, in_channels, out_channels, downsampling = 1, bias = False, 
                 padding = 1):
        super().__init__()
        self.g_conv = P4ConvZ2(in_channels, out_channels, stride = downsampling, 
                               bias = bias, padding = padding)

    def forward(self,x):
        x = self.g_conv(x)
        return x



# ------------------------ CLASS RESIDUAL BLOCK --------------------------------
# In:  pre-activated 5D array: [batch_size, n_channels_in, n_transformations, height_in, width_in]
# Out: pre-activated 5D array: [batch_size, n_channels_out, n_transformations,  height_out, width_out]

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks   = nn.Identity() #initialize block as identity
        self.shortcut = nn.Identity() #initialize skip as identity
    
    def forward(self, x):
        #shortcut
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)

        #combination
        x += residual
        x = F.relu(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels


# ----------------------- CLASS RESNET RESIDUAL BLOCK --------------------------
class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, 
                 conv=P4ConvP4, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv

        self.shortcut = nn.Sequential(OrderedDict({
            'conv' : P4ConvP4(self.in_channels, 
                                self.expanded_channels, kernel_size=1,
                                stride=self.downsampling, bias=False),
            'bn'   : nn.BatchNorm3d(self.expanded_channels)
            
        })) if self.should_apply_shortcut else None 

        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels



# --------------------- PRE CONVOLUTION SEQUENTIAL -----------------------------
def bn_conv(in_channels, out_channels, conv, activation = nn.ReLU, *args, **kwargs):
    return nn.Sequential(OrderedDict({'conv': conv(in_channels, out_channels, 
                                                   kernel_size = 3, padding = 1, 
                                                   *args, **kwargs),
                                      'bn': nn.BatchNorm3d(out_channels)}))
    

# ------------------------- CLASS BASIC BLOCK ----------------------------------
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            bn_conv(self.in_channels, self.out_channels, conv=self.conv, 
                     bias=False, stride=self.downsampling),
            nn.ReLU(),
            bn_conv(self.out_channels, self.expanded_channels, 
                     conv=self.conv, bias=False),
        )


# ------------------------- CLASS RESNET LAYER ---------------------------------
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        #first introduce block(... )
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            # in the next the n-1 blocks are subsequently stacked (_ underscore means
            # that the do_something will be executed the prescribed amount of times
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x


# ------------------------- CLASS RESNET ENCODER -------------------------------
class ResNetEncoder(nn.Module): 
    def __init__(self, in_channels=3, blocks_sizes=[16, 32, 64], depths=[7,7,7], 
                 activation=nn.ReLU, block=ResNetBasicBlock, *args,**kwargs):
        super().__init__()

        self.blocks_sizes = blocks_sizes
        
        self.initconv = nn.Sequential(OrderedDict({
            'conv'    : P4ConvZ2(in_channels, self.blocks_sizes[0], 
                        kernel_size=7, stride = 1, padding=3, bias=False),
            'bn'      : nn.BatchNorm3d(self.blocks_sizes[0]),
            'act'     : activation()})
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=depths[0], activation=activation, 
                        block=block,  *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, depths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.initconv(x)
        for block in self.blocks:
            x = block(x)
        return x


# ------------------------- CLASS RESNET DECODER -------------------------------
class ResNetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes, *args, **kwargs):
        super().__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avgpool(x)
        x = x.view(x.size(0), -1) #flatten
        x = self.decoder(x)
        return x


# ------------------------------ CLASS RESNET ----------------------------------
class ResNet(nn.Module):
    '''
    This class represents the full implementation of the residual network. This is
    the connection of the encoder (initial convolution and residual stages) 
    followed by the decoder (fully connected network)
    '''
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()

        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResNetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels*4, n_classes)

    def forward(self, x):
        x = self.encoder(x)
        xs = x.size()
        x = x.view(xs[0], xs[1]*xs[2], xs[3], xs[4])
        x = self.decoder(x)
        return x


# ----------------------------- DEFINE RESNET44 --------------------------------
def resnet44_P4(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, depths=[7, 7, 7])

def arbitraryresnet(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, depths=[3, 3, 3])

def test():
    x = Variable(torch.randn(50,3,32,32))
    net = resnet44_P4(3,10)
    y = net(x)
    print(y.size())

test()

torch.Size([50, 10])


#Summary of the Model

In [0]:
#G-CNN group p4
model_p4 = resnet44_P4(3, 10)
summary(model_p4.cuda(), (3, 32, 32))

#Loading Data

In [0]:
#Normalize a tensor image with mean and standard deviation. 
# Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input torch.*Tensor i.e. input[channel] = (input[channel] - mean[channel]) / std[channel]
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load training sets
batch_size  = 32
num_workers = 2
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

# Load test sets
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)


Files already downloaded and verified
Files already downloaded and verified


#Training+Testing (code)

In [0]:
# ------------------------- TRAINING THE NETWORK -------------------------------
def train_network(net, train_loader, test_loader, device, title,
                  lr = 0.05, momentum = 0.9, gamma = 0.1, 
                  n_epochs = 300, epoch_start = 0,
                  criterion = nn.CrossEntropyLoss().cuda()):
    """
    Training and evaluation of a connected network which should be written
    in Pytorch fashion.
    """

    print('training on', device)

    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

    #Create vectors with information on loss, training accuracy and test accuracy
    loss_curve   = np.zeros((n_epochs))
    train_curve  = np.zeros((n_epochs))
    test_curve   = np.zeros((n_epochs))

    #vector with epochs
    epoch_vec = np.arange(epoch_start,n_epochs)

    # ---------------------------- START TRAINING ------------------------------
    for epoch in epoch_vec:

        net.train()
        n, start = 0, time.time()

        train_l_sum   = torch.tensor([0.0], dtype=torch.float32, device=device)
        train_acc_sum = torch.tensor([0.0], dtype=torch.float32, device=device)

        for i, data in enumerate(train_loader, 0):

            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            with torch.no_grad():
                outputs = outputs.long()
                train_l_sum += loss.float()
                n += outputs.shape[0]

        #update learning rate
        if epoch <= 300:
            if epoch % 50 == 49:
                lr *= gamma
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
                print('We got ourselves a change in learning rate:')
                print('Epoch: '+ str(epoch))
                print('Learning rate changed to: ' + str(lr))

        # ------------------- START VALIDATION and SAVING ----------------------
        loss_value = train_l_sum/n

        stop1 = time.time()
        train_acc  = evaluate_accuracy(train_loader, net, device)
        stop2 = time.time()
        test_acc   = evaluate_accuracy(test_loader, net, device)

        loss_curve[epoch]  = loss_value
        train_curve[epoch] = train_acc
        test_curve[epoch]  = test_acc

        curves = np.array([loss_curve, train_curve, test_curve])

        #print updated scores
        print('epoch %d [#]  |  loss %.4f [-]  |  train acc %.3f [-]  |  test acc %.3f [-]  |  time %.1f-%.1f-%.1f [s]'\
        % (epoch + 1, loss_value, train_acc, test_acc, stop1-start, stop2-stop1, time.time()-stop2))
        
        #save every 5 epochs
        if epoch % 50 == 49:    # save every 10 epochs
            #model
            name_to_save = title + '__test=' + str(test_acc) + '%_train=' + str(train_acc) + '%_epoch=' + str(epoch+1)
            directory = '/content/gdrive/My Drive/Deep Learning/Reproduction project/Saved Networks/Preliminary/'
            path = '/content/gdrive/My Drive/Deep Learning/Reproduction project/Saved Networks/Preliminary/' + name_to_save + '.pth'
            torch.save(model.state_dict(), path)
            
            #learning curves
            path2 = directory + name_to_save + '_start:' + str(epoch_start)
            np.save(path2, curves) 
      
    print('Finished Training')
    return loss_curve, train_curve, test_curve



# ---------------------------------- VALIDATION --------------------------------
def evaluate_accuracy(data_iter, net, device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  # Switch to evaluation mode for Dropout, BatchNorm etc layers.
    acc_sum, n = torch.tensor([0], dtype=torch.float32, device=device), 0
    for X, y in data_iter:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0]
    return acc_sum.item()/n



# --------------------------- WEIGHT INITIALIZATION ----------------------------
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)



# ---------------------------- TRYING GPU of COLAB -----------------------------
def try_gpu():
    """If GPU is available, return torch.device as cuda:0; else return torch.device as cpu."""
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

#Training+Testing(evaluation)

In [0]:
device = try_gpu()

model = resnet44_P4(3,10)
model.cuda()

#name_model = input('Name model?')
#path_model = '/content/gdrive/My Drive/Deep Learning/Reproduction project/Saved Networks/Preliminary/' + name_model + '.pth'
#model = resnet44_P4(3, 10)
#result = model.load_state_dict(torch.load(path_model))
#model.cuda()

#pre-trained pytorch ResNet50
#model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet50', pretrained=True)

epoch_start = 0
lr = 0.05

bs = input('What is the batch size:')
title = 'ResNet44Groupy10_bs:' + bs +'_SGD_every50epochs'
loss_curve, train_curve, test_curve = train_network(model, trainloader, testloader, device, title, n_epochs = 300, epoch_start = epoch_start, lr=lr)

training on cuda:0
epoch 1 [#]  |  loss 0.0690 [-]  |  train acc 0.286 [-]  |  test acc 0.288 [-]  |  time 148.8-53.1-10.8 [s]
epoch 2 [#]  |  loss 0.0555 [-]  |  train acc 0.407 [-]  |  test acc 0.408 [-]  |  time 147.9-52.8-10.7 [s]
epoch 3 [#]  |  loss 0.0481 [-]  |  train acc 0.489 [-]  |  test acc 0.487 [-]  |  time 149.3-54.2-10.7 [s]
epoch 4 [#]  |  loss 0.0415 [-]  |  train acc 0.574 [-]  |  test acc 0.564 [-]  |  time 151.0-53.4-10.8 [s]
epoch 5 [#]  |  loss 0.0338 [-]  |  train acc 0.681 [-]  |  test acc 0.665 [-]  |  time 147.8-52.6-10.8 [s]
epoch 6 [#]  |  loss 0.0273 [-]  |  train acc 0.720 [-]  |  test acc 0.688 [-]  |  time 148.0-52.8-10.6 [s]
epoch 7 [#]  |  loss 0.0227 [-]  |  train acc 0.783 [-]  |  test acc 0.743 [-]  |  time 147.2-52.4-10.5 [s]
epoch 8 [#]  |  loss 0.0194 [-]  |  train acc 0.795 [-]  |  test acc 0.739 [-]  |  time 146.8-52.9-10.7 [s]
epoch 9 [#]  |  loss 0.0163 [-]  |  train acc 0.850 [-]  |  test acc 0.771 [-]  |  time 148.1-53.1-10.8 [s]
epoch 10 

KeyboardInterrupt: ignored

#Loading Model

In [0]:
#Loading model
# BatchSize=16_NONpre__test=61.13%_train=66.414%_epoch=20
device = try_gpu()
load_model = input('Load model? [y/n]:  ')

if load_model == 'y':
    name_model = input('Name model?')
    path_model = '/content/gdrive/My Drive/Deep Learning/Reproduction project/Saved Networks/Preliminary/' + name_model + '.pth'
    model = ResNet(3, 10, block=ResNetBasicBlock, depths=[7, 7, 7])
    result = model.load_state_dict(torch.load(path_model)) #check of alle weights zijn geladen
    print('')
    print('=== Weights loaded ===')
    print('Missing keys:   ', result.missing_keys)
    print('Unexpected keys:', result.unexpected_keys)

model.cuda()

#check accuracy

check = input('Do you want to check accuracy? [y/n]:  ')
if check == 'y':
    print('')
    print('Çheck training (first row) and test (second row) error:')
    print(evaluate_accuracy(trainloader, model, device=device))
    print(evaluate_accuracy(testloader, model, device=device))

Load model? [y/n]y
Name model?BatchSize=16_NONpre__test=61.13%_train=66.414%_epoch=20

=== Weights loaded ===
Missing keys:    []
Unexpected keys: []

Çheck training (first row) and test (second row) error:
0.66414
0.6113
